<a href="https://colab.research.google.com/github/Ali-mohammadi-design/ML-practice/blob/main/Prompt_engineering_Fine_tunning_Medical_Text_dataset_Data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [2]:
import openai
print(openai.__version__)

1.6.1


In [4]:
import os
API=os.environ['openAI_API_Key']

In [6]:
from openai import OpenAI
client= OpenAI(api_key=API)

We want to fine tune the medical report including the name of the disease and medical emergencies as well as their description.

 The very first step is data preprocessing or data cleaning

Data PreProcessing:

In [8]:
import pandas as pd
medical_reports=pd.read_csv('/reports_copy.csv')

In [9]:
medical_reports.head()

,medical_specialty,report
0,Cardiovascular / Pulmonary,"2-D M-MODE: , ,1. Left atrial enlargement wit..."
1,Cardiovascular / Pulmonary,1. The left ventricular cavity size and wall ...
2,Cardiovascular / Pulmonary,"2-D ECHOCARDIOGRAM,Multiple views of the heart..."
3,Cardiovascular / Pulmonary,"DESCRIPTION:,1. Normal cardiac chambers size...."
4,Cardiovascular / Pulmonary,"2-D STUDY,1. Mild aortic stenosis, widely calc..."


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('report').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_2.sort_values('index', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('report')):
  _plot_series(series, series_name, i)
  fig.legend(title='report', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_3['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_4['report'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_4, x='index', y='report', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [10]:
medical_reports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2179 entries, 0 to 2178
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   medical_specialty  2179 non-null   object
 1   report             2179 non-null   object
dtypes: object(2)
memory usage: 34.2+ KB


Removing the nulls in the report section:

Note: We can use fillna to fill the nulls

In [12]:
medical_reports= medical_reports.dropna(subset=['report'])

In [13]:
medical_reports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2179 entries, 0 to 2178
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   medical_specialty  2179 non-null   object
 1   report             2179 non-null   object
dtypes: object(2)
memory usage: 34.2+ KB


In the next step we have to split the data to train, test, and validation

In [14]:
medical_reports.groupby('medical_specialty').count()

,report
medical_specialty,
Cardiovascular / Pulmonary,371
Gastroenterology,224
Neurology,223
Radiology,273
Surgery,1088


Note: .groupby() would group all the specific parameters in a context (here is medical specialty). The we would seperate the specific amount of sample from each of the groups.

In [23]:
grouped_data= medical_reports.groupby('medical_specialty').sample(110, random_state=42)

In [25]:
grouped_data['medical_specialty'].value_counts()

Cardiovascular / Pulmonary    110
Gastroenterology              110
Neurology                     110
Radiology                     110
Surgery                       110
Name: medical_specialty, dtype: int64

Now in each group we have 110 data. For every group we could separate 10 data for validation and test (5 for validation and 5 for test).

In [34]:
val_test_data= grouped_data.groupby('medical_specialty').sample(10, random_state=42)

In [36]:
val_data= val_test_data.groupby('medical_specialty').head(5)
test_data= val_test_data.groupby('medical_specialty').tail(5)

Now we have to remove the test and validation data from the main data. We can use the .index.isin() function.

Index.isin(values, level=None)

Return a boolean array where the index values are in values.


note: ~ means not

note2: .index would derermine the position of the data in the list

In [38]:
train= grouped_data[~grouped_data.index.isin(val_test_data.index)]

In [39]:
len(train)

500

reviewing statistics and estimating the pricing!

In [40]:
! pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [41]:
import tiktoken

In [42]:
def num_tokens_from_string(string):
  encoding=tiktoken.get_encoding('cl100k_base')
  num_token=len(encoding.encode(string))
  return num_token

In [45]:
num_tokens_from_string("hello world, how are you?")

7

.Apply()

Apply a function along an axis of the DataFrame.

Objects passed to the function are Series objects whose index is either the DataFrame’s index (axis=0) or the DataFrame’s columns (axis=1). apply function to each column., the final return type is inferred from the return type of the applied function

In [57]:
report_length= train['report'].apply(num_tokens_from_string)

In [59]:
report_length.describe()

count     500.000000
mean      674.788000
std       427.809075
min        14.000000
25%       358.000000
50%       593.500000
75%       904.000000
max      3701.000000
Name: report, dtype: float64

In [67]:
number_of_tokens=report_length.sum()
number_of_tokens

337394

Thus, in total we have 337394 tokens!!!

in the next stage we would calculate the pricing

In [65]:
price_per_1000_Token_training= 0.008/1000

In [66]:
final_price=price_per_1000_Token_training*number_of_tokens
final_price

2.6991519999999998